# Pandas 🐼🐼🐼

### Tipici usi della libreria

| Esigenza tipica                                                  | Funzionalità Pandas                     | Quando serve                                                                |
| ---------------------------------------------------------------- | --------------------------------------- | --------------------------------------------------------------------------- |
| 1. **Importare** formati eterogenei (CSV, Excel, TSV, JSON, SQL) | `read_*()`                              | Ogni volta che i dati provengono da laboratori, EHR, registry o altri dispositivi |
| 2. **Pulire** dati sporchi o parziali                            | `dropna`, `fillna`, `astype`, `replace` | Prima di qualsiasi statistica o di usarli per Machine Learning                                          |
| 3. **Ristrutturare** il dataset (modificarne la struttura per facilitarne l’analisi o per adattarlo a specifiche esigenze)                                  | `melt`, `pivot`, `stack/unstack`        | Per allineare o sistemare misure longitudinali, matrici di espressione...                 |
| 4. **Filtrare & creare sottogruppi**                                    | Boolean indexing, `query`, `groupby`    | Focus su un fenotipo, un trattamento, un gene...                               |
| 5. **Aggregare & statistiche rapide**                            | `groupby().agg()`, `describe`           | Report clinici, cruscotti QC (Quality Control)...                                                |
| 6. **Merge/Join** di più fonti                                   | `merge`, `concat`                       | Integrare imaging + clinica + omics (Dati omici → Genomica, trascrittomica, proteomica, metabolomica)                                         |
| 7. **Time series per il monitoraggio continuo**                                               | `to_datetime`, `resample`, `rolling`    | Wearable, follow-up, Analisi di segnali continui in terapia intensiva (ICU waveform analysis)...                                           |


### Perché non basta usare liste Python o array NumPy al posto di Pandas?

| Esigenza tipica nei dati biomedici                     | Liste Python                                | NumPy array                                           | Pandas DataFrame                                |
| ------------------------------------------------------ | ------------------------------------------- | ----------------------------------------------------- | ----------------------------------------------- |
| **Etichette (ID paziente, nome gene, tempo)**          | ❌ gestite “a mano” con strutture ausiliarie | ❌ non esistono; solo indici interi                    | ✅ `index`, `columns`, gerarchici (`MultiIndex`) |
| **Dati eterogenei** (numeri + stringhe + date)         | ✅ ma zero funzioni vettoriali               | ⚠️ possibile con `dtype=object`, ma perde performance | ✅ colonne con tipi diversi, ottimizzate         |
| **Valori mancanti** (`NaN`, `None`, sentinel)          | ❌ serve logica ad-hoc                       | ⚠️ `np.nan` solo per float; complicato per tipi misti | ✅ `isna`, `fillna`, propagazione coerente       |
| **Operazioni “SQL-like”** (`groupby`, `join`, `pivot`) | ❌ codice manuale e ciclo esplicito          | ❌ non previste                                        | ✅ una riga di codice idiomatica                 |
| **Time series** (resample, rolling window)             | ❌ da implementare                           | ⚠️ serve `np.searchsorted`/for loop                   | ✅ API dedicate (`resample`, `rolling`, `shift`) |
| **I/O nativo** (CSV, Excel, SQL, Parquet)              | ❌ librerie esterne, parsing manuale         | ❌ idem                                                | ✅ `read_*`, `to_*` one-liner                    |
| **Leggibilità notebook/paper**                         | 😐 spesso 3-4 strutture parallele           | 😐 array “senza volto”                                | 😀 tabelle auto-formattate                      |


### 🏥 1 | Analisi di dataset clinici multifattoriali
(= studiare dati sanitari considerando più variabili contemporaneamente, come età, sesso, genetica, stile di vita, parametri fisiologici ...)

Scenario
Cartella clinica con: età, sesso, BMI, pressione S/D, HbA1c, diagnosi, outcome.

In [1]:
import pandas as pd
df = pd.read_csv("registry_diabete.csv")
df["diagnosi"] = df["diagnosi"].str.upper().str.strip()  # normalizzo
stats = (df.groupby("diagnosi")[["HbA1c", "pressione_S", "pressione_D"]]
           .mean()
           .round(2))

ModuleNotFoundError: No module named 'pandas'

Utilità:
- Pulizia nomenclatura diagnosi
- Statistiche descrittive per reparto
- Individuazione outlier (df[df["HbA1c"]>12])
- Esportazione rapida in Excel per il board clinico (stats.to_excel("summary.xlsx"))

{Tabella heat-map dei valori medi di HbA1c per diagnosi}

### 🧬 2 | Trascrittomica / RNA-seq
Scenario
Matrice genes × samples (>30 000 × 200).

In [ ]:
expr = pd.read_parquet("rna_seq.parquet")  # veloce su big data
# Filtra geni espressi
expr = expr.loc[expr.mean(axis=1) > 1]
corr = expr.T.corr(method="spearman")

Utilità:
- Lettura in Parquet → RAM-efficient
- Filtri basati su media/varianza
- Calcolo di reti di co-espressione
- Output diretto verso Seaborn/Scanpy

### 🧠 3 | Imaging e Quantificazione
Scenario
CSV export da 3D Slicer: ID paziente, ROI, volume (mm³), densità (HU).

In [ ]:
img = pd.read_csv("slicer_ROI_metrics.csv")
lesioni = img.query("volume_mm3 > 500 & ROI == 'tumor'")
trend = (lesioni.groupby("patient_id")
                 .apply(lambda g: g.sort_values("scan_date")
                                    .assign(delta=g["volume_mm3"].diff())))


Utilità:
- Filtraggio rapido su soglia volumetrica
- Calcolo delta-volume fra follow-up
- Facilita il merge con outcome clinici

### 🧫 4 | High-Content Screening (96-well, 384-well)
Scenario
Assorbanza OD a 0 h, 24 h, 48 h per 300 composti.

In [ ]:
raw = pd.read_excel("HCS_readout.xlsx")
pivot = raw.pivot_table(values="OD",
                        index=["compound_id"],
                        columns=["time_h"])
pivot["growth_%"] = (pivot[48] - pivot[0]) / pivot[0] * 100
top_hits = pivot.nlargest(10, "growth_%")

Utilità
- pivot_table rende il dataset “wide” per calcolo Δ
- Rank dei top-hits per successiva validazione
- Integrare annotazioni chimiche con merge

{Foto di piastra 384-well con pozzi colorimetrici + barplot crescita}

### 💉 5. Studi di Coorte Longitudinali
Scenario:
Dataset con pazienti seguiti per 5 anni, misurando outcome clinici (es. progressione di malattia, sopravvivenza, esiti).

In [ ]:
df["data_visita"] = pd.to_datetime(df["data_visita"])
df.sort_values(["paziente_id", "data_visita"], inplace=True)


cohort = pd.read_csv("coorte_cardiologia.csv", parse_dates=["visit_date"])
cohort.sort_values(["patient_id", "visit_date"], inplace=True)
cohort["years_from_baseline"] = (cohort["visit_date"] -
                                 cohort.groupby("patient_id")["visit_date"].transform("first")
                                ).dt.days / 365.25

Utilità:
- Ordinamento e calcolo di intervalli temporali
- Analisi per paziente o sottogruppo
- Calcolo della progressione (delta valori nel tempo)

{Grafico temporale con andamento di biomarcatori in un paziente nel tempo}

# Numpy

NumPy is a Python library that provides a simple yet powerful data structure: the n-dimensional array.

**Perché scegliere NumPy**
Pur conoscendo già Python “puro” (con i suoi cicli `for`, la lettura/scrittura di CSV, ecc.), NumPy introduce un paradigma che offre vantaggi concreti:

1. **Maggiore velocità**

   * NumPy utilizza algoritmi implementati in C, che eseguono operazioni in nanosecondi anziché in secondi.
2. **Riduzione dei cicli**

   * Grazie alle strutture array, è possibile comporre operazioni vettoriali che eliminano gran parte dei loop manuali e l’indice di iterazione.
3. **Codice più leggibile**

   * Senza cicli annidati, le espressioni nel codice assomigliano molto di più alle equazioni matematiche che si vogliono calcolare.
4. **Alta qualità e affidabilità**

   * Un’ampia comunità di sviluppatori mantiene NumPy veloce, di facile utilizzo e privo di bug.

**Conclusione**
Questi fattori hanno reso NumPy lo standard “de facto” per gli array multidimensionali in Python applicato alla data science. Molte librerie popolari si basano su NumPy: impararlo fornisce una solida base su cui poi sviluppare competenze più avanzate in ambiti specifici.


In [ ]:
%%bash
# attiviamo l'ambiente virutale
. .venv/bin/activate
# Installiamo numpy
pip install numpy

  Using cached numpy-2.2.5-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.5-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.1 MB)


In [ ]:
%%bash
# installiamo matplotlib
pip install numpy matplotlib